In [1]:
pip install riotwatcher

In [2]:
from riotwatcher import LolWatcher, ApiError

In [3]:
# Creates the list to contain our players and stats
def openStatList(myFriends):
    players = []
    for name in myFriends:
        addPlayer(players, name)
    return players

# Adding empty stat players 
def addPlayer(players, name, gamesPlayed = 0, gamesWon = 0):
    player = {
        'name': name,
        'gamesPlayed': gamesPlayed,
        'gamesWon': gamesWon,
    }
    players.append(player)

# Saves all our players' puuids for easy access 
def cacheAccounts(friendList):
    puuidNameDict = {}
    for name in friendList:
        puuid = getPuuidFromName(name)
        puuidNameDict.update({puuid: name})
    return puuidNameDict

def getPuuidFromName(name):
    summonerdto = lol_watcher.summoner.by_name(region, name)
    return summonerdto['puuid']

# Returns only our friends' names from the match
def getNamesFromMatch(gamedetails, accountsDict):
    nameList = []
    puuidList = gamedetails['metadata']['participants'] # gets list of all 10 players
    for puuid in puuidList:
        if puuid in accountsDict:
            nameList.append(accountsDict.get(puuid))
    return nameList

# Returns whether we've won the game or not
def hasWon(gamedetails, teamList):
    for i in range(10):
        if gamedetails['info']['participants'][i]['summonerName'] in teamList:
            return (gamedetails['info']['participants'][i]['win'])
    return

# Update player stats
def updateStatList(statList, players, victory):
    for player in players:
        for i in range(len(statList)):
            if player == statList[i]['name']:
                statList[i]['gamesPlayed'] += 1
                if (victory):
                    statList[i]['gamesWon'] += 1
                break
    return statList

def displayStats(statList):
    finaltext = ""
    for i in range(len(statList)):
        winratetext = str(round((statList[i]['gamesWon']/statList[i]['gamesPlayed']*100),0))
        gamesPlayedtext = str(statList[i]['gamesPlayed'])
        if statList[i]['gamesPlayed'] != 0:
            if statList[i]['name'] != playerName:
                finaltext += statList[i]['name']+": "+winratetext+"% of "+gamesPlayedtext+" games. \n"
            else:
                print(playerName+" has a winrate of "+winratetext+"% of "+gamesPlayedtext+" games, and have the following winrates with:")
    print(finaltext)

In [4]:
# Make sure to register/renew apiKey at: https://developer.riotgames.com/ 
apiKey = 'RGAPI-XX'

# setup riot-watcher with our key
lol_watcher = LolWatcher(apiKey)
region = 'oc1'

In [5]:
# change if needed
playerName = "Fwoqqet"
myFriends = [
    playerName,
    'pinkfolder2', 
    'XxElitePetexX',
    'Floppy Corn',
    'Elliechu',
    'kdng',
    'BirdBrainn',
    'HoChiLim',
    'PhotonBlade8'
]

# grab my puuid
me = lol_watcher.summoner.by_name(region, playerName)
puuid = me['puuid']

# grab all our friends' puuids- so we don't have to constantly search for them
statList = openStatList(myFriends)
puuidNameDict = cacheAccounts(myFriends)

# grab the matches
index = 0
matches = 100
queueid = 450 #this is ARAM
games = lol_watcher.match.matchlist_by_puuid(region, puuid, 0, matches, queueid)

for gameid in games:
    gamedetails = lol_watcher.match.by_id(region, gameid)
    namesList = getNamesFromMatch(gamedetails, puuidNameDict)
    victory = hasWon(gamedetails, namesList)
    statList = updateStatList(statList, namesList, victory)
    
displayStats(statList)

Elliechu has a winrate of 43.0% of 100 games, and have the following winrates with:
pinkfolder2: 48.0% of 44 games. 
XxElitePetexX: 44.0% of 63 games. 
Floppy Corn: 49.0% of 69 games. 
Fwoqqet: 82.0% of 11 games. 
kdng: 44.0% of 9 games. 
BirdBrainn: 55.0% of 22 games. 
HoChiLim: 37.0% of 30 games. 
PhotonBlade8: 50.0% of 12 games. 

